In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/calcofi/bottle.csv
/kaggle/input/calcofi/cast.csv


In [56]:
# modules we'll use
import pandas as pd
import numpy as np

# for Box-Cox Transformation
from scipy import stats

# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt

# read in all our data
bottle = pd.read_csv("../input/calcofi/bottle.csv")

# set seed for reproducibility
np.random.seed(0)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (47,73) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
bottle.head()

,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,...,R_PHAEO,R_PRES,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1,DIC Quality Comment
0,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.50,33.440,NaN,25.649,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.46,33.440,NaN,25.656,NaN,...,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.46,33.437,NaN,25.654,NaN,...,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.45,33.420,NaN,25.643,NaN,...,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.45,33.421,NaN,25.643,NaN,...,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
bottle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864863 entries, 0 to 864862
Data columns (total 74 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Cst_Cnt              864863 non-null  int64  
 1   Btl_Cnt              864863 non-null  int64  
 2   Sta_ID               864863 non-null  object 
 3   Depth_ID             864863 non-null  object 
 4   Depthm               864863 non-null  int64  
 5   T_degC               853900 non-null  float64
 6   Salnty               817509 non-null  float64
 7   O2ml_L               696201 non-null  float64
 8   STheta               812174 non-null  float64
 9   O2Sat                661274 non-null  float64
 10  Oxy_µmol/Kg          661268 non-null  float64
 11  BtlNum               118667 non-null  float64
 12  RecInd               864863 non-null  int64  
 13  T_prec               853900 non-null  float64
 14  T_qual               23127 non-null   float64
 15  S_prec           

we should drop features with 30% or more missing values

In [59]:
bottle.drop(['BtlNum', 'DIC Quality Comment', 
             'pH1', 'pH2', 'TA2', 'TA1', 'DIC2',
             'DIC1', 'R_SAMP', 'R_PHAEO', 'R_CHLA',
             'R_NO2', 'R_NO3', 'R_PO4', 'R_SIO3',
             'LightP', 'IncTim', 'BtlNum', 'T_qual',
             'S_qual', 'SThtaq', 'ChlorA', 'Phaeop',
             'PO4uM', 'PO4uM', 'PO4q', 'SiO3uM',
             'SiO3qu', 'NO2uM', 'NO2q', 'NO3uM', 'NO3q',
             'C14As1', 'C14A1p', 'DarkAs', 'DarkAp', 
             'R_NH4', 'NH3uM', 'C14As2', 'MeanAs', 'MeanAp',
             'C14A2p', 'O2Satq', 'O_qual'], axis = 1, inplace = True)

In [60]:
bottle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864863 entries, 0 to 864862
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Cst_Cnt      864863 non-null  int64  
 1   Btl_Cnt      864863 non-null  int64  
 2   Sta_ID       864863 non-null  object 
 3   Depth_ID     864863 non-null  object 
 4   Depthm       864863 non-null  int64  
 5   T_degC       853900 non-null  float64
 6   Salnty       817509 non-null  float64
 7   O2ml_L       696201 non-null  float64
 8   STheta       812174 non-null  float64
 9   O2Sat        661274 non-null  float64
 10  Oxy_µmol/Kg  661268 non-null  float64
 11  RecInd       864863 non-null  int64  
 12  T_prec       853900 non-null  float64
 13  S_prec       817509 non-null  float64
 14  P_qual       673755 non-null  float64
 15  Chlqua       639166 non-null  float64
 16  Phaqua       639170 non-null  float64
 17  NH3q         808299 non-null  float64
 18  C14A1q       848605 non-

In [61]:
bottle.describe()

,Cst_Cnt,Btl_Cnt,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,RecInd,...,R_Depth,R_TEMP,R_POTEMP,R_SALINITY,R_SIGMA,R_SVA,R_DYNHT,R_O2,R_O2Sat,R_PRES
count,864863.000000,864863.000000,864863.000000,853900.000000,817509.000000,696201.000000,812174.000000,661274.000000,661268.000000,864863.000000,...,864863.000000,853900.000000,818816.000000,817509.000000,812007.000000,812092.000000,818206.000000,696201.000000,666448.000000,864863.000000
mean,17138.790958,432432.000000,226.831951,10.799677,33.840350,3.392468,25.819394,57.103779,148.808694,4.700273,...,226.832495,10.799697,10.842295,33.840295,25.811010,220.938918,0.431763,3.392498,57.192940,228.395694
std,10240.949817,249664.587267,316.050259,4.243825,0.461843,2.073256,1.167787,37.094137,90.187533,1.877428,...,316.050007,4.243829,4.236435,0.462978,1.124645,93.483703,0.382336,2.073271,37.060537,319.456731
min,1.000000,1.000000,0.000000,1.440000,28.431000,-0.010000,20.934000,-0.100000,-0.434900,3.000000,...,0.000000,1.440000,0.000000,4.570000,20.934000,0.400000,0.000000,-0.010000,-0.100000,0.000000
25%,8269.000000,216216.500000,46.000000,7.680000,33.488000,1.360000,24.965000,21.100000,60.915470,3.000000,...,46.000000,7.680000,7.740000,33.488000,24.960000,143.700000,0.130000,1.360000,21.200000,46.000000
50%,16848.000000,432432.000000,125.000000,10.060000,33.863000,3.440000,25.996000,54.400000,151.064150,3.000000,...,125.000000,10.060000,10.100000,33.863000,25.990000,203.200000,0.340000,3.440000,54.500000,126.000000
75%,26557.000000,648647.500000,300.000000,13.880000,34.196900,5.500000,26.646000,97.600000,240.379600,7.000000,...,300.000000,13.880000,13.920000,34.196000,26.640000,299.800000,0.640000,5.500000,97.600000,302.000000
max,34404.000000,864863.000000,5351.000000,31.140000,37.034000,11.130000,250.784000,214.100000,485.701800,7.000000,...,5351.000000,31.140000,31.140000,37.034000,250.784000,683.400000,3.880000,11.130000,214.100000,5458.000000


in preparation of data for our machine learning model i advise we fill all our numerical data with -999 so it would help take care or outliers

In [62]:
def get_all_unique_values(dataframe):
  for cols in dataframe.columns:
    print(f"{cols}")
    print(dataframe[cols].unique())
    print(len(dataframe[cols].unique()))
    print("=" * 80)

get_all_unique_values(bottle)

Cst_Cnt
[    1     2     3 ... 34402 34403 34404]
34404
Btl_Cnt
[     1      2      3 ... 864861 864862 864863]
864863
Sta_ID
['054.0 056.0' '052.0 075.0' '051.0 085.0' ... '067.0 050.0' '067.0 047.7'
 '067.1 047.7']
2634
Depth_ID
['19-4903CR-HY-060-0930-05400560-0000A-3'
 '19-4903CR-HY-060-0930-05400560-0008A-3'
 '19-4903CR-HY-060-0930-05400560-0010A-7' ...
 '20-1611SR-MX-310-2239-09340264-0005A-3'
 '20-1611SR-MX-310-2239-09340264-0010A-3'
 '20-1611SR-MX-310-2239-09340264-0015A-3']
864850
Depthm
[   0    8   10 ... 3499 2001 3501]
3219
T_degC
[10.5   10.46  10.45  ... 18.692 18.161 17.533]
13848
Salnty
[33.44   33.437  33.42   ... 33.7432 33.6689 33.4218]
14603
O2ml_L
[  nan 5.9   6.02  ... 3.761 2.997 6.355]
6501
STheta
[25.649   25.656   25.654   ... 23.88911 24.01426 24.15297]
36369
O2Sat
[   nan  92.8  111.8  ...  67.88  30.55 108.46]
10380
Oxy_µmol/Kg
[     nan 256.9761 262.6628 ... 123.19    48.35    31.34  ]
391815
RecInd
[3 7 5 6 4]
5
T_prec
[ 1.  2. nan  3.]
4
S_prec
[ 2.  3.

the code cell above helps us to see all the uniqe values in each of our features and that helps us determin which is categorical and which is not

In [72]:
id_col = ["Depth_ID"]

#Target columns
target_col = []

#categorical columns
cat_cols = bottle.nunique()[bottle.nunique() < 6].keys().tolist() # To filter out categorical columns
cat_cols = [x for x in cat_cols if x not in target_col] # To remove the target column

#numerical columns
num_cols = [x for x in bottle.columns if x not in cat_cols + target_col + id_col]

#Binary columns
bin_cols = bottle.nunique()[bottle.nunique() == 2].keys().tolist() 

#Columns with more than two values
multi_cols = [i for i in cat_cols if i not in bin_cols]

cat_cols

['RecInd',
 'T_prec',
 'S_prec',
 'P_qual',
 'Chlqua',
 'Phaqua',
 'NH3q',
 'C14A1q',
 'C14A2q',
 'DarkAq',
 'MeanAq']

In [64]:
for col in num_cols:
    bottle[col].fillna(-999,  inplace = True)

In [65]:
bottle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864863 entries, 0 to 864862
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Cst_Cnt      864863 non-null  int64  
 1   Btl_Cnt      864863 non-null  int64  
 2   Sta_ID       864863 non-null  object 
 3   Depth_ID     864863 non-null  object 
 4   Depthm       864863 non-null  int64  
 5   T_degC       864863 non-null  float64
 6   Salnty       864863 non-null  float64
 7   O2ml_L       864863 non-null  float64
 8   STheta       864863 non-null  float64
 9   O2Sat        864863 non-null  float64
 10  Oxy_µmol/Kg  864863 non-null  float64
 11  RecInd       864863 non-null  int64  
 12  T_prec       853900 non-null  float64
 13  S_prec       817509 non-null  float64
 14  P_qual       673755 non-null  float64
 15  Chlqua       639166 non-null  float64
 16  Phaqua       639170 non-null  float64
 17  NH3q         808299 non-null  float64
 18  C14A1q       848605 non-

then i would fill the categorical features with the mode

In [73]:
for cols in cat_cols:
    bottle[cols].fillna(bottle[cols].mode()[0], inplace=True)

In [74]:
bottle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864863 entries, 0 to 864862
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Cst_Cnt      864863 non-null  int64  
 1   Btl_Cnt      864863 non-null  int64  
 2   Sta_ID       864863 non-null  object 
 3   Depth_ID     864863 non-null  object 
 4   Depthm       864863 non-null  int64  
 5   T_degC       864863 non-null  float64
 6   Salnty       864863 non-null  float64
 7   O2ml_L       864863 non-null  float64
 8   STheta       864863 non-null  float64
 9   O2Sat        864863 non-null  float64
 10  Oxy_µmol/Kg  864863 non-null  float64
 11  RecInd       864863 non-null  int64  
 12  T_prec       853900 non-null  float64
 13  S_prec       817509 non-null  float64
 14  P_qual       673755 non-null  float64
 15  Chlqua       639166 non-null  float64
 16  Phaqua       639170 non-null  float64
 17  NH3q         808299 non-null  float64
 18  C14A1q       848605 non-